### 0. Requirements

- My Drive에 업로드해야 할 폴더 및 파일 <br/> : checkpoints 폴더, kpfbert 폴더, kpfSBERT 폴더, predict_module.py

In [ ]:
# google colab에서 실행 시
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install --no-cache-dir numpy

In [ ]:
pip install pandas scipy swifter

In [ ]:
pip install torch torchvision torchaudio

In [ ]:
pip install transformers sentencepiece

In [ ]:
pip install pymysql sqlalchemy

In [ ]:
pip install pytorch-lightning

In [ ]:
pip install kss

In [ ]:
pip install gensim

In [ ]:
pip install konlpy

In [ ]:
pip install sentence-transformers

In [ ]:
import os
os._exit(00)

In [ ]:
import os
import re
import sys
import kss
import torch
import swifter
import pymysql
import logging
import sqlalchemy
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from dotenv import load_dotenv
from konlpy.tag import Okt
from collections import Counter
from torch.nn.init import xavier_uniform_
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from sentence_transformers import SentenceTransformer, util
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from sqlalchemy import text
from sqlalchemy import create_engine, text
from sqlalchemy.dialects.mysql import insert
from sklearn.model_selection import train_test_split
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess

In [ ]:
# Google Drive 경로 추가(predice_module.py 경로 지정 위함)
sys.path.append("/content/drive/My Drive")

In [ ]:
# 뉴스 기사 요약 - KPF-BERTSum model
from predict_module import summarize_test

[Kss]: GPU available: False, used: False
[Kss]: TPU available: False, using: 0 TPU cores
[Kss]: HPU available: False, using: 0 HPUs


In [ ]:
# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"현재 디바이스: {device}")

In [ ]:
# KPF-SBERT 모델 로드(google drive)
model_path = "/content/drive/MyDrive/kpfSBERT"
if not os.path.exists(model_path):
    logging.error(f"모델 경로 없음: {model_path}")
    exit()

model = SentenceTransformer(model_path).to(device)

NameError: name 'os' is not defined

### 1. 전처리

In [ ]:
# MySQL 연결 설정
db_url_mysql = os.getenv("MYSQL_URL")
engine = create_engine(db_url_mysql)

In [ ]:
# 기존 `cleaned_news_data` 테이블에서 데이터 불러오기
query_existing = "SELECT id FROM cleaned_news_data"
df_existing = pd.read_sql(query_existing, engine)

In [ ]:
# 기존 ID 목록
existing_ids = set(df_existing["id"].tolist())

In [ ]:
# 새로 가져온 뉴스 데이터
query_new = "SELECT id_org, title_org, pub_date_org, newspaper_org, content_org, link_org FROM news_data"
df = pd.read_sql(query_new, engine)

In [ ]:
# 기존에 없는 새로운 데이터만 필터링
df_new = df[~df["id_org"].isin(existing_ids)].copy()

if df_new.empty:
    print("새로운 데이터가 없습니다.")
else:
    print(f"새로운 데이터 개수: {len(df_new)}")

In [ ]:
# 불필요한 단어 목록
UNWANTED_PATTERNS = [
    # 'ㅇㅇㅇ기자' 형태 제거
    r"\b\w+기자\b",
    r"\b\w{2,5}기자\b",
    r"\b\w+\s*기자\b",
    # '구독 구독중' 제거
    r"구독\s*구독중",
    # '이전 다음 이미지확대' 제거
    r"이전\s*다음\s*이미지확대",
    # '이전 다음' 제거
    r"이전\s*다음",
    # '이전 다음 이미지 확대' 제거
    r"이전\s*다음\s*이미지\s*확대?",
    # '동영상 고정 취소' 제거
    r"동영상\s*고정\s*취소",
    # '사진게티이미지' 제거
    r"사진게티이미지",
    r"사진\s*게티이미지",
    r"사진\s*이베이",
    # 'viewer' 제거
    r"viewer",
    # '사진 확대' 제거
    r"사진\s*확대",
    # 읽어주기 기능 관련 메시지 제거
    r"읽어주기\s*기능은\s*크롬기반의\s*브라우저에서만\s*사용하실\s*수\s*있습니다\s*.",
    # '전체재생' 제거
    r"전체재생"
    # "서울경제, 무단 전재 및 재배포 금지" 단독 문구 제거
    r"서울경제,\s*무단\s*전재\s*및\s*재배포\s*금지",
    # "저작권자 ⓒ 신문사명, 무단 전재 및 재배포 금지" 문구 삭제
    r"<*\s*저작권자\s*ⓒ?\s*[가-힣A-Za-z\s]+,\s*무단\s*전재\s*및\s*재배포\s*금지\s*>*",
    # 모든 '사진 제공 OO' 패턴 삭제 (괄호 포함 경우도 고려)
    r"\(?사진\s*제공\s*[=:]?\s*[가-힣A-Za-z]+\)?",
    # "사진 제공" 다음 단어가 OO일 경우 제거 (ex: "사진 제공 경기도")
    r"\b사진 제공 \b[가-힣A-Za-z]+\b",
    r"전체재생",
    r"앵커",
    r"사진=게티이미지뱅크",
    r"사진=연합뉴스",
    r"\(사진\)",
    r"한 눈에 읽기",
    r"뉴스 요약쏙 AI 요약은 OpenAI의 최신 기술을 활용해 핵심 내용을 빠르고 정확하게 제공합니다\. 전체 맥락을 이해하려면 기사 본문을 함께 확인하는 것이 좋습니다\.",
    r"공유",
    r"이메일",
    r"기사저장",
    r"이 기사와 관련된 기사",
    r"무단전재재배포 금지",
    r"저작권자 파이낸셜뉴스",
    r"AD 투자가를 위한 경제콘텐츠 플랫폼",
    r"무단전재 배포금지",
    r"아시아경제www\.asiae\.co\.kr",
    r"파이낸셜뉴스",
    r"사진뉴스1",
    r"연합뉴스",
    r"제보하기",
    r"채널 추가 전화 \d{4,}",
    r"김광수 특파원의 ‘중알중알’은 ‘중국을 알고 싶어 중국을 알려줄게’의 줄임말입니다\..*?구독을 하시면 매주 금요일 유익한 중국 정보를 전달받으실 수 있습니다\.",
    r"헤럴드경제",
    r"\*\s*편집자\s*주:\s*‘AI\s*PRISM’.*?제공합니다\.",
    r"\*\s*편집자\s*주\s*:\s*‘AI\s*PRISM’.*?제공합니다\.",
]

: 

In [ ]:
# 본문 전처리 함수
def clean_text(text):
    for pattern in UNWANTED_PATTERNS:
        text = re.sub(pattern, "", text)
        
    # 특수문자 제거(마침표, 쉼표, 작은 따옴표, 큰 따옴표, 퍼센트센트 유지)
    text = re.sub(r"[^가-힣a-zA-Z0-9.,'\"%p\s‘’“”△:()]", "", text)
    # 숫자와 단위(조, 원, %, p) 사이 공백 유지
    text = re.sub(r"(\d+)\s+(조|원|%|p)", r"\1\2", text)
    # 연속공백 제거
    text = re.sub(r"\s+", " ", text).strip()
    # 대괄호([]) 포함 내용 제거
    text = re.sub(r"\[.*?\]", "", text)
    # 온점(`.`) 뒤에 공백 추가 (이미 공백이 있으면 유지)
    text = re.sub(r"\.(?!\s|$)", ". ", text)
    # `△`를 쉼표(`,`)로 변경하고 앞 공백 제거, 뒤에 공백 추가
    text = re.sub(r"\s*△\s*", ", ", text)
    
    return text

# 본문 전처리 적용
df_new["content_display"] = df_new["content_org"].apply(clean_text)

In [ ]:
# 제목 전처리 함수
def clean_title(text):
    for pattern in UNWANTED_PATTERNS:
        text = re.sub(pattern, "", text)

    # 대괄호([]) 포함 내용 삭제
    text = re.sub(r"\[.*?\]", "", text)
    # 특수문자 제거
    ttext = re.sub(r"[^가-힣a-zA-Z0-9.,'\"%p\s‘’“”:()]", "", text)
    # 연속공백 제거
    text = re.sub(r"\s+", " ", text).strip()

    return text


# 제목 전처리 적용
df_new["title"] = df_new["title_org"].apply(clean_title)

In [12]:
# 중복된 id_org 값 확인
duplicate_ids = df["id_org"].duplicated().sum()
print(f"중복된 ID 개수: {duplicate_ids}")

중복된 ID 개수: 0


In [ ]:
# 필요한 컬럼 선택 및 NULL 값 추가
df_insert = df[["id_org", "title_org", "title", "pub_date_org", "newspaper_org", "content_display", "link_org"]].copy()
df_insert["content_summary"] = ""  # 추후 데이터 추가 예정
df_insert["keyword_5"] = ""  # 추후 데이터 추가 예정
df_insert["keyword_MMR"] = ""  # 추후 데이터 추가 예정
df_insert["sentiment"] = None # 추후 데이터 추가 예정
df_insert["sentiment_gpt"] = None # 추후 데이터 추가 예정

# 컬럼명 변경
df_insert.rename(columns={"id_org": "id", "pub_date_org": "pub_date"}, inplace=True)

# 데이터 삽입
with engine.begin() as conn:
    df_insert.to_sql("cleaned_news_data", conn, if_exists="append", index=False)


### 2. 형태소 분석 및 키워드 추출

In [ ]:
# 컬럼 불러오기
## keyword_5(코사인 유사도 계산)와 keyword_MMR(MMR 알고리즘) 변경
query = "SELECT id, title, content_display, keyword_5 FROM cleaned_news_data WHERE keyword_5 = ''"
df = pd.read_sql(query, engine)  # 빈 문자열 값이 있는 데이터만 불러오기


# 데이터 개수 체크
print("쿼리 결과 개수:", len(df))
if df.empty:
    print("업데이트할 데이터가 없습니다. 실행을 중단합니다.")
    exit()

In [47]:
# 형태소 분석기 초기화
okt = Okt()

In [ ]:
# 불용어 리스트
STOPWORDS = {"없다", "없는", "없습니다", "않다", "아무", "없이", "없다는", "없었다", "모르다"}

In [ ]:
# < 코사인 유사도 기반 키워드 추출 - keyword_5 >
def extract_keywords_with_embedding(title, content, top_n=5, threshold=0.4):
    # 본문이 None 또는 NaN이면 빈 리스트 반환
    if pd.isna(content) or not isinstance(content, str) or content.strip() == "":
        return ["국내 자동차"]

    words = [word for word, pos in okt.pos(content) if pos in ["Noun"] and len(word) > 1 and word not in STOPWORDS]

    # 출현 빈도를 고려하여 중요한 단어 우선 선택
    word_freq = Counter(words)
    sorted_words = [word for word, freq in word_freq.most_common(50)]  # 최대 50개 단어 사용

    if not sorted_words:
        return ["국내 자동차"]

    # 제목과 키워드를 한 번에 벡터화하여 성능 최적화
    sentences = [title] + sorted_words
    embeddings = model.encode(sentences, convert_to_tensor=True, device="cuda")

    title_embedding = embeddings[0].cpu()
    word_embeddings = embeddings[1:].cpu()

    similarities = util.pytorch_cos_sim(title_embedding, word_embeddings).squeeze(0)
    filtered_keywords = [
        sorted_words[i] for i in similarities.argsort(descending=True).tolist() if similarities[i] >= threshold
    ]

    unique_keywords = list(dict.fromkeys(filtered_keywords))[:top_n]
    unique_keywords = [word for word in unique_keywords if not word.replace("%", "").isdigit()]
    if not unique_keywords:
        return ["국내 자동차"]

    return unique_keywords

# 키워드 추출 실행 (병렬 처리 적용)
df["keyword_5"] = df.swifter.apply(
    lambda row: ", ".join(
        extract_keywords_with_embedding(str(row["title"]), str(row["content_display"]), top_n=5, threshold=0.4)
    ), 
    axis=1
)

# `None` 또는 NaN을 빈 문자열("")로 변경 (최종 보정)
df["keyword_5"] = df["keyword_5"].fillna("").astype(str)

# 업데이트할 데이터 리스트 생성 (`None`을 빈 문자열로 대체)
update_data = [
    {"keyword": row["keyword_5"] if row["keyword_5"] else "", "id": row["id"]}
    for _, row in df.iterrows()
]

# `update_data`가 비어있는 경우 실행 안 함
if not update_data:
    print("업데이트할 데이터가 없습니다. 실행을 중단합니다.")
    sys.exit(0) 

print("업데이트할 데이터 샘플:", update_data[:5])  # 일부 데이터 출력

# 키워드 저장 - 기존 테이블 업데이트
sql =  text("""
        UPDATE cleaned_news_data
        SET keyword_5 = :keyword
        WHERE id = :id
        """)

# SQL 실행 최적화
with engine.begin() as conn:
    conn.execute(sql, update_data) 

print("`keyword_5` 컬럼 업데이트 완료!")

StatementError: (sqlalchemy.exc.InvalidRequestError) A value is required for bind parameter 'keyword'
[SQL: 
        UPDATE cleaned_news_data
        SET keyword_5 = %(keyword)s
        WHERE id = %(id)s
        ]
(Background on this error at: https://sqlalche.me/e/20/cd3x)

In [ ]:
# 컬럼 불러오기
## keyword_5(코사인 유사도 계산)와 keyword_MMR(MMR 알고리즘) 변경
query = "SELECT id, title, content_display, keyword_MMR FROM cleaned_news_data WHERE keyword_MMR = ''"
df = pd.read_sql(query, engine)  # 빈 문자열 값이 있는 데이터만 불러오기


# 데이터 개수 체크
print("쿼리 결과 개수:", len(df))
if df.empty:
    print("업데이트할 데이터가 없습니다. 실행을 중단합니다.")
    exit()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# 형태소 분석기 초기화
okt = Okt()

In [ ]:
# 불용어 리스트
STOPWORDS = {"없다", "없는", "없습니다", "않다", "아무", "없이", "없다는", "없었다", "모르다"}

In [ ]:
# < MMR(Maximal Marginal Relevance) 알고리즘 기반 키워드 추출 - keyword_MMR >

def extract_keywords_with_mmr(title, content, top_n=5, diversity=0.7):
    # 본문이 None 또는 NaN이면 빈 리스트 반환
    if pd.isna(content) or not isinstance(content, str) or content.strip() == "":
        return ["국내 자동차"]

    words = [word for word, pos in okt.pos(content) if pos in ["Noun"] and len(word) > 1 and word not in STOPWORDS]

    # 출현 빈도를 고려하여 중요한 단어 우선 선택 - 최대 50개
    word_freq = Counter(words)
    sorted_words = [word for word, freq in word_freq.most_common(50)]

    # 단어가 1개 이하라면 기본 키워드 반환
    if len(sorted_words) < 2:
        return ["국내 자동차"]

    # 제목과 키워드를 한 번에 벡터화하여 성능 최적화
    sentences = [title] + sorted_words
    embeddings = model.encode(sentences, convert_to_tensor=True, device=device)

    title_embedding = embeddings[0]
    word_embeddings = embeddings[1:]

    # 제목과 단어 간 유사도 계산(Relevance)
    word_similarities = util.pytorch_cos_sim(title_embedding, word_embeddings).squeeze(0)

    # MMR 알고리즘 적용
    selected_keywords = []
    selected_indices = set()

    for _ in range(min(top_n, len(sorted_words))):
        candidates = [
            (idx, word_similarities[idx].item())
            for idx in range(len(sorted_words))
            if idx not in selected_indices
        ]

        # 선택할 단어가 없을 시 종료
        if not candidates:
            break

        if not selected_keywords:
            # Relevance 기준 가장 유사한 키워드 선택
            best_idx = max(candidates, key=lambda x: x[1])[0]
        else:
            # Diversity를 고려하여 MMR Score 계산
            if selected_indices:  # 선택된 키워드가 있을 때만 Diversity 계산
                similarity_scores = torch.stack(
                    [util.pytorch_cos_sim(word_embeddings[i], word_embeddings).squeeze(0) for i in selected_indices]
                )
                diversity_score = torch.mean(similarity_scores, dim=0)  # 평균 계산
            else:
                diversity_score = torch.zeros_like(word_similarities)

            mmr_score = diversity * word_similarities - (1 - diversity) * diversity_score

            # 벡터에서 값 추출 시 `.item()` 적용 방식 수정
            mmr_candidates = [
                (idx, float(mmr_score[idx]))  # 텐서에서 값 추출
                for idx in range(len(sorted_words))
                if idx not in selected_indices
            ]

            best_idx = max(mmr_candidates, key=lambda x: x[1])[0]

        selected_indices.add(best_idx)
        selected_keywords.append(sorted_words[best_idx])

    # 중복 제거 후 반환
    return list(set(selected_keywords)) if selected_keywords else ["국내 자동차"]

# 기존 cleaned_news_data 테이블에서 id, title, content_display 컬럼 불러오기
query = """
SELECT id, title, content_display, keyword_MMR
FROM cleaned_news_data
WHERE keyword_MMR = '' OR keyword_MMR IS NULL
"""
df = pd.read_sql(query, engine)

# 병렬 처리로 MMR 키워드 추출 적용
df["keyword_MMR"] = df.swifter.apply(
    lambda row: ", ".join(
        extract_keywords_with_mmr(str(row["title"]), str(row["content_display"]), top_n=5, diversity=0.7)
    ),
    axis=1
)

# MySQL에 저장
sql = text("""
            UPDATE cleaned_news_data
            SET keyword_MMR = :keyword_MMR
            WHERE id = :id
            """)

update_data = []
for _, row in df.iterrows():
    keyword_mmr = row["keyword_MMR"]

    # 빈 문자열이면 기본값 "국내 자동차"로 변경
    if keyword_mmr == "" or keyword_mmr is None or pd.isna(keyword_mmr):
        keyword_mmr = "국내 자동차"

    update_data.append({"keyword_MMR": keyword_mmr, "id": row["id"]})

# 디버깅용 업데이트할 데이터 개수 출력
print(f"업데이트할 데이터 개수: {len(update_data)}")

if update_data:
    with engine.begin() as conn:
        conn.execute(sql, update_data)

    print("keyword_MMR 컬럼 업데이트 완료!")

- 키워드 추출 성능 평가 지표 <br/> 뉴스 주제별 연관된 키워드를 선택하고 싶으면 코사인 유사도 방식 <br/> 더 다양한 키워드를 제공하려면 MMR 방식

In [ ]:
# 키워드 추출 성능 평가 지표
## 키워드 다양성 평가(Diversity Score) - 키워드 간 평균 코사인 유사도 측정. 낮을수록 MMR 효과적.

# Sentence-BERT 모델 로드
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

def calculate_diversity_score(keyword_list):
    """
    키워드 리스트를 입력받아 다양성을 평가하는 함수.
    코사인 유사도의 평균값을 Diversity Score로 사용.
    """
    if len(keyword_list) < 2:
        return 1.0  # 키워드가 1개 이하일 경우, 다양성이 없는 것으로 간주하여 1.0 반환

    # 키워드 임베딩 생성
    keyword_embeddings = model.encode(keyword_list, convert_to_tensor=True)
    
    # 키워드 간 코사인 유사도 계산
    similarity_matrix = util.pytorch_cos_sim(keyword_embeddings, keyword_embeddings)

    # 상삼각행렬(triu)에서 대각선 제외 후 평균 계산
    num_keywords = len(keyword_list)
    diversity_score = (
        torch.sum(similarity_matrix) - torch.sum(torch.diagonal(similarity_matrix))
    ) / (num_keywords * (num_keywords - 1))

    return diversity_score.item()  # 낮을수록 다양한 키워드가 선택됨

# 예제 키워드 비교
keywords_cosine = ["자동차", "전기차", "내연기관", "배터리", "충전"]
keywords_mmr = ["자동차", "수출", "시장", "친환경", "신차"]

print(f"코사인 유사도 기반 키워드 다양성 점수: {calculate_diversity_score(keywords_cosine):.4f}")
print(f"MMR 기반 키워드 다양성 점수: {calculate_diversity_score(keywords_mmr):.4f}")

### 3. 기사 요약

In [ ]:
########################
# GPU 환경(Google Colab)
########################

In [ ]:
##########################
# 함께 드라이브에 업로드
# predict_module.py 파일
# kpfbert 파일
# checkpoints 폴더
##########################

In [ ]:
# KPF-BERT 모델 로드(google drive)
model_path = "/content/drive/MyDrive/kpfbert"

In [ ]:
# 경로가 제대로 설정되었는지 확인
if os.path.exists(model_path):
    print(f"BERT 모델 경로 확인 완료: {model_path}")
else:
    print(f"BERT 모델 경로 오류: {model_path}가 존재하지 않습니다.")

In [ ]:
# 모델 로드
tokenizer = BertTokenizer.from_pretrained(model_path)
bert_model = BertModel.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")
print("BERT 모델 및 Tokenizer 로드 성공")

In [ ]:
# MySQL 연결 설정
db_url_mysql = os.getenv("MYSQL_URL")
engine = create_engine(db_url_mysql, pool_recycle=3600)

In [ ]:
# MySQL 연결 테스트
try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT COUNT(*) FROM cleaned_news_data"))
        count = result.scalar()  
        print(f"데이터 개수: {count} 개")
except Exception as e:
    print(f"MySQL 연결 실패: {e}")

In [ ]:
# hyper parameter 설정
MAX_TOKEN_COUNT = 512
N_EPOCHS = 20
BATCH_SIZE = 8

- Data

In [ ]:
# MySQL에서 뉴스 기사 본문 데이터 가져오기
query = "SELECT content_display FROM cleaned_news_data"
df = pd.read_sql(query, con=engine)

In [63]:
# 결측치 제거
df = df.dropna()

print(f"전체 데이터 개수 {len(df)}")

전체 데이터 개수 1084


In [ ]:
# 데이터 분할 - 학습/검증/테스트
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [69]:
# 인덱스 초기화
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [ ]:
# 테스트 데이터(test_df) 확보 (최대 100개)
test_df = val_df[:100] if len(val_df) > 100 else val_df.copy()
val_df = val_df[100:] if len(val_df) > 100 else val_df.copy()

In [72]:
# 데이터 크기 출력
print(f"train_df.shape(학습): {train_df.shape}, test_df.shape(테스트): {test_df.shape}, val_df.shape(검증): {val_df.shape}")

train_df.shape(학습): (867, 1), test_df.shape(테스트): (100, 1), val_df.shape(검증): (117, 1)


- Preprocess

In [ ]:
def preprocess_data(data):
    outs = []
    for idx, row in data.iterrows():
        # 문장 단위 분리
        article_original = kss.split_sentences(row['content_display'])
        outs.append([article_original])

    return outs

In [ ]:
# 병렬 처리 문장 분리
import multiprocessing

def split_sentences_parallel(texts):
    with multiprocessing.Pool(processes=os.cpu_count()) as pool:
        result = pool.map(kss.split_sentences, texts)
    return result

In [ ]:
# 적용 # 경고 메시지 무시 가능
train_df['article_original'] = split_sentences_parallel(train_df['content_display'])
test_df['article_original'] = split_sentences_parallel(test_df['content_display'])
val_df['article_original'] = split_sentences_parallel(val_df['content_display'])

print("데이터 전처리 완료!")

- Tokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_path)

- Dataset(Presumm에서 제안한 형식으로 인코딩 - bert에서 여러 문장을 입력하기 위해)

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class SummDataset(Dataset): # 데이터셋 presumm 방식 인코딩

    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: BertTokenizer,
        max_token_len: int = 512
    ):
        self.tokenizer = tokenizer
        self.data = data
        self.max_token_len = max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        tokenlist = []
        for sent in data_row.article_original:
            tokenlist.append(self.tokenizer(
                text = sent,
                add_special_tokens = True))

        src = [] # 토크나이징 된 전체 문단
        segs = []  #각 토큰에 대해 홀수번째 문장이면 0, 짝수번째 문장이면 1을 매핑
        clss = []  #[CLS]토큰의 포지션값을 지정
        labels = [] #문장별 요약 여부(요약이면 1, 아니면 0)

        odd = 0
        for tkns in tokenlist:
            if odd > 1:
                odd = 0
            clss.append(len(src))
            src.extend(tkns['input_ids'])
            segs.extend([odd] * len(tkns['input_ids']))

            # labels 추가
            if 'extractive' in data_row and isinstance(data_row.extractive, list):
                labels.append(1 if tokenlist.index(tkns) in data_row.extractive else 0)
            else:
                labels.append(0)

            odd += 1

            #truncation
            if len(src) >= self.max_token_len:
                src = src[:self.max_token_len - 1] + [src[-1]]
                segs = segs[:self.max_token_len]
                break

        #padding
        pad_len = self.max_token_len - len(src)
        src.extend([0] * pad_len)
        segs.extend([0] * pad_len)
        clss.extend([-1] * (self.max_token_len - len(clss)))
        labels.extend([0] * (self.max_token_len - len(labels)))

        return dict(
            src=torch.tensor(src),
            segs=torch.tensor(segs),
            clss=torch.tensor(clss),
            labels=torch.tensor(labels, dtype=torch.float)
        )

In [ ]:
class SummDataModule(pl.LightningDataModule): # presumm 인코딩 모듈

    def __init__(self, train_df, test_df, val_df, tokenizer, batch_size=1, max_token_len=512):
        super().__init__()
        self.batch_size = batch_size
        self.train_df = train_df
        self.test_df = test_df
        self.val_df = val_df
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len

    def setup(self, stage=None):
        self.train_dataset = SummDataset(self.train_df, self.tokenizer, self.max_token_len)
        self.test_dataset = SummDataset(self.test_df, self.tokenizer, self.max_token_len)
        self.val_dataset = SummDataset(self.val_df, self.tokenizer, self.max_token_len)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=0)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=0)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=0)

In [ ]:
# DataModule 생성
data_module = SummDataModule(
    train_df,
    test_df,
    val_df,
    tokenizer,
    batch_size=BATCH_SIZE,
    max_token_len=MAX_TOKEN_COUNT
)

print("데이터 로더 생성 완료!")

In [ ]:
data_module.setup(stage="fit")

In [ ]:
# train_dataloader()에서 labels 포함 여부 확인
sample_loader = iter(data_module.train_dataloader())
sample_batch = next(sample_loader)

print(sample_batch.keys())  # 'labels'가 포함되어 있는지 확인
print(sample_batch['labels'].shape)  # labels의 형태 확인

- Model <br/> (kpfBERT를 pretrained_bert로 불러와서 후처리 레이어 추가 후 문장 추출 모델 만듦)

In [ ]:
import torch.nn as nn
import math

In [ ]:
class PositionalEncoding(nn.Module): # positional embedding

    def __init__(self, dropout, dim, max_len=5000):
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp((torch.arange(0, dim, 2, dtype=torch.float) *
                            -(math.log(10000.0) / dim)))
        pe[:, 0::2] = torch.sin(position.float() * div_term)
        pe[:, 1::2] = torch.cos(position.float() * div_term)
        pe = pe.unsqueeze(0)
        super(PositionalEncoding, self).__init__()
        self.register_buffer('pe', pe)
        self.dropout = nn.Dropout(p=dropout)
        self.dim = dim

    def forward(self, emb, step=None):
        emb = emb * math.sqrt(self.dim)
        if (step):
            emb = emb + self.pe[:, step][:, None, :]

        else:
            emb = emb + self.pe[:, :emb.size(1)]
        emb = self.dropout(emb)
        return emb

    def get_emb(self, emb):
        return self.pe[:, :emb.size(1)]

In [ ]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, heads, d_ff, dropout):
        super(TransformerEncoderLayer, self).__init__()

        self.self_attn = MultiHeadedAttention(
            heads, d_model, dropout=dropout)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, iter, query, inputs, mask):
        if (iter != 0):
            input_norm = self.layer_norm(inputs)
        else:
            input_norm = inputs

        mask = mask.unsqueeze(1)
        context = self.self_attn(input_norm, input_norm, input_norm, mask=mask)
        out = self.dropout(context) + inputs
        return self.feed_forward(out)

In [ ]:
class ExtTransformerEncoder(nn.Module):
    def __init__(self, hidden_size=768, d_ff=2048, heads=8, dropout=0.2, num_inter_layers=2):
        super(ExtTransformerEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_inter_layers = num_inter_layers
        self.pos_emb = PositionalEncoding(dropout, hidden_size)
        self.transformer_inter = nn.ModuleList(
            [TransformerEncoderLayer(hidden_size, heads, d_ff, dropout)
            for _ in range(num_inter_layers)])
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(hidden_size, eps=1e-6)
        self.wo = nn.Linear(hidden_size, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, top_vecs, mask):
        """ See :obj:`EncoderBase.forward()`"""

        batch_size, n_sents = top_vecs.size(0), top_vecs.size(1)
        pos_emb = self.pos_emb.pe[:, :n_sents]
        x = top_vecs * mask[:, :, None].float()
        x = x + pos_emb

        for i in range(self.num_inter_layers):
            x = self.transformer_inter[i](i, x, x, ~mask.bool())

        x = self.layer_norm(x)
        sent_scores = self.sigmoid(self.wo(x))
        sent_scores = sent_scores.squeeze(-1) * mask.float()

        return sent_scores

In [ ]:
class PositionwiseFeedForward(nn.Module):
    """ A two-layer Feed-Forward-Network with residual layer norm.

    Args:
        d_model (int): the size of input for the first-layer of the FFN.
        d_ff (int): the hidden layer size of the second-layer
            of the FNN.
        dropout (float): dropout probability in :math:`[0, 1)`.
    """

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)

    def gelu(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))


    def forward(self, x):
        inter = self.dropout_1(self.gelu(self.w_1(self.layer_norm(x))))
        output = self.dropout_2(self.w_2(inter))
        return output + x

In [ ]:
class MultiHeadedAttention(nn.Module):
    """
    Multi-Head Attention module from
    "Attention is All You Need"
    :cite:`DBLP:journals/corr/VaswaniSPUJGKP17`.

    Similar to standard `dot` attention but uses
    multiple attention distributions simulataneously
    to select relevant items.

    .. mermaid::

    graph BT
        A[key]
        B[value]
        C[query]
        O[output]
        subgraph Attn
            D[Attn 1]
            E[Attn 2]
            F[Attn N]
        end
        A --> D
        C --> D
        A --> E
        C --> E
        A --> F
        C --> F
        D --> O
        E --> O
        F --> O
        B --> O

    Also includes several additional tricks.

    Args:
    head_count (int): number of parallel heads
    model_dim (int): the dimension of keys/values/queries,
        must be divisible by head_count
    dropout (float): dropout parameter
    """

    def __init__(self, head_count, model_dim, dropout=0.1, use_final_linear=True):
        assert model_dim % head_count == 0
        self.dim_per_head = model_dim // head_count
        self.model_dim = model_dim

        super(MultiHeadedAttention, self).__init__()
        self.head_count = head_count

        self.linear_keys = nn.Linear(model_dim,
                                     head_count * self.dim_per_head)
        self.linear_values = nn.Linear(model_dim,
                                       head_count * self.dim_per_head)
        self.linear_query = nn.Linear(model_dim,
                                      head_count * self.dim_per_head)
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)
        self.use_final_linear = use_final_linear
        if (self.use_final_linear):
            self.final_linear = nn.Linear(model_dim, model_dim)

    def forward(self, key, value, query, mask=None,
                layer_cache=None, type=None, predefined_graph_1=None):
        """
        Compute the context vector and the attention vectors.

        Args:
        key (`FloatTensor`): set of `key_len`
                key vectors `[batch, key_len, dim]`
        value (`FloatTensor`): set of `key_len`
                value vectors `[batch, key_len, dim]`
        query (`FloatTensor`): set of `query_len`
                query vectors  `[batch, query_len, dim]`
        mask: binary mask indicating which keys have
                non-zero attention `[batch, query_len, key_len]`
        Returns:
        (`FloatTensor`, `FloatTensor`) :

           * output context vectors `[batch, query_len, dim]`
           * one of the attention vectors `[batch, query_len, key_len]`
        """

        batch_size = key.size(0)
        dim_per_head = self.dim_per_head
        head_count = self.head_count
        key_len = key.size(1)
        query_len = query.size(1)

        def shape(x):
            """  projection """
            return x.view(batch_size, -1, head_count, dim_per_head) \
                .transpose(1, 2)

        def unshape(x):
            """  compute context """
            return x.transpose(1, 2).contiguous() \
                .view(batch_size, -1, head_count * dim_per_head)

        # 1) Project key, value, and query.
        if layer_cache is not None:
            if type == "self":
                query, key, value = self.linear_query(query), \
                                    self.linear_keys(query), \
                                    self.linear_values(query)

                key = shape(key)
                value = shape(value)

                if layer_cache is not None:
                    device = key.device
                    if layer_cache["self_keys"] is not None:
                        key = torch.cat(
                            (layer_cache["self_keys"].to(device), key),
                            dim=2)
                    if layer_cache["self_values"] is not None:
                        value = torch.cat(
                            (layer_cache["self_values"].to(device), value),
                            dim=2)
                    layer_cache["self_keys"] = key
                    layer_cache["self_values"] = value
            elif type == "context":
                query = self.linear_query(query)
                if layer_cache is not None:
                    if layer_cache["memory_keys"] is None:
                        key, value = self.linear_keys(key), \
                                    self.linear_values(value)
                        key = shape(key)
                        value = shape(value)
                    else:
                        key, value = layer_cache["memory_keys"], \
                                    layer_cache["memory_values"]
                    layer_cache["memory_keys"] = key
                    layer_cache["memory_values"] = value
                else:
                    key, value = self.linear_keys(key), \
                                self.linear_values(value)
                    key = shape(key)
                    value = shape(value)
        else:
            key = self.linear_keys(key)
            value = self.linear_values(value)
            query = self.linear_query(query)
            key = shape(key)
            value = shape(value)

        query = shape(query)

        key_len = key.size(2)
        query_len = query.size(2)

        # 2) Calculate and scale scores.
        query = query / math.sqrt(dim_per_head)
        scores = torch.matmul(query, key.transpose(2, 3))

        if mask is not None:
            mask = mask.unsqueeze(1).expand_as(scores)
            scores = scores.masked_fill(mask.bool(), -1e18)

        # 3) Apply attention dropout and compute context vectors.

        attn = self.softmax(scores)

        if (not predefined_graph_1 is None):
            attn_masked = attn[:, -1] * predefined_graph_1
            attn_masked = attn_masked / (torch.sum(attn_masked, 2).unsqueeze(2) + 1e-9)

            attn = torch.cat([attn[:, :-1], attn_masked.unsqueeze(1)], 1)

        drop_attn = self.dropout(attn)
        if (self.use_final_linear):
            context = unshape(torch.matmul(drop_attn, value))
            output = self.final_linear(context)
            return output
        else:
            context = torch.matmul(drop_attn, value)
            return context

In [ ]:
class Summarizer(pl.LightningModule): # 요약 모델

    def __init__(self, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        self.max_pos = 512
        self.bert = BertModel.from_pretrained(model_path, add_pooling_layer=False)
        self.ext_layer = ExtTransformerEncoder()
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.loss = nn.BCELoss(reduction='none')
        self.training_losses = []  # 학습 손실 저장
        self.validation_losses = []  # 검증 손실 저장
        self.test_losses = []  # 테스트 손실 저장

        for p in self.ext_layer.parameters():
            if p.dim() > 1:
                xavier_uniform_(p)

    def forward(self, src, segs, clss, labels=None): #, input_ids, attention_mask, labels=None):

        mask_src = ~(src == 0).bool() # 1 - (src == 0)
        mask_cls = ~(clss == -1).bool() # 1 - (clss == -1)

        top_vec = self.bert(src, token_type_ids=segs, attention_mask=mask_src)
        top_vec = top_vec.last_hidden_state

        sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss]
        sents_vec = sents_vec * mask_cls[:, :, None].float()

        sent_scores = self.ext_layer(sents_vec, mask_cls).squeeze(-1)

        loss = 0
        if labels is not None:
            loss = self.loss(sent_scores, labels)
            loss = (loss * mask_cls.float()).sum() / len(labels)

        return loss, sent_scores

    def step(self, batch):
        src = batch['src']
        if len(batch['labels']) > 0 :
            labels = batch['labels']
        else:
            labels = None
        segs = batch['segs']
        clss = batch['clss']

        loss, sent_scores = self(src, segs, clss, labels)

        return loss, sent_scores, labels

    def training_step(self, batch, batch_idx):

        loss, sent_scores, labels = self.step(batch)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        self.training_losses.append(loss.item()) # 손실 저장

        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def validation_step(self, batch, batch_idx):

        loss, sent_scores, labels = self.step(batch)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        self.validation_losses.append(loss.item())

        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def test_step(self, batch, batch_idx):

        loss, sent_scores, labels = self.step(batch)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        self.test_losses.append(loss.item())

        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def on_train_epoch_end(self):
        avg_loss = sum(self.training_losses) / len(self.training_losses)
        print(f"Epoch {self.current_epoch} - Train Loss: {avg_loss}")
        self.log("avg_train_loss", avg_loss, prog_bar=True, logger=True)
        self.training_losses.clear()  # 손실 리스트 초기화

    def on_validation_epoch_end(self):
        avg_loss = sum(self.validation_losses) / len(self.validation_losses)
        print(f"Epoch {self.current_epoch} - Validation Loss: {avg_loss}")
        self.log("avg_val_loss", avg_loss, prog_bar=True, logger=True)
        self.validation_losses.clear()  # 손실 리스트 초기화

    def on_test_epoch_end(self):
        avg_loss = sum(self.test_losses) / len(self.test_losses)
        print(f"Test Loss: {avg_loss}")
        self.log("avg_test_loss", avg_loss, prog_bar=True, logger=True)
        self.test_losses.clear()  # 손실 리스트 초기화

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=2e-5)

        if 'train_df' in globals():
            steps_per_epoch = len(train_df) // BATCH_SIZE
        else:
            steps_per_epoch = 1  # 기본값 지정

        total_training_steps = steps_per_epoch * N_EPOCHS

        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=steps_per_epoch,
            num_training_steps=total_training_steps
        )

        return dict(
            optimizer=optimizer,
            lr_scheduler=dict(
                scheduler=scheduler,
                interval='step'
            )
        )

In [ ]:
model = Summarizer().to(device)

- Training

In [ ]:
# Google Colab environment
!rm -rf /content/lightning_logs/
!rm -rf /content/checkpoints/

In [ ]:
# TensorBoard 실행
%load_ext tensorboard
%tensorboard --logdir /content/lightning_logs

In [ ]:
# checkpoint 저장 설정
checkpoint_callback = ModelCheckpoint(
    # Colab에서의 절대 경로
    dirpath="/content/checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="avg_val_loss",
    mode="min"
)

In [ ]:
# TensorBoard logger 설정
logger = TensorBoardLogger("/content/lightning_logs", name="kpfBERT_Summary")

In [ ]:
# 조기 종료 설정
early_stopping_callback = EarlyStopping(
    monitor='avg_val_loss',
    patience=10,
    verbose=True,
    mode='min'
    )

In [ ]:
# Trainer 설정
trainer = pl.Trainer(
    logger=logger,
    callbacks=[checkpoint_callback, early_stopping_callback],
    max_epochs=N_EPOCHS,
    accelerator="gpu",
    devices=1, # GPU 개수
    enable_progress_bar=True
)

print("Trainer 설정 완료")

In [ ]:
# Trainer 실행
trainer.fit(model, data_module)

In [ ]:
sample_loader = iter(data_module.train_dataloader())
sample_batch = next(sample_loader)

print(sample_batch.keys())  # 'labels'가 포함되어 있는지 확인
print(sample_batch['labels'].shape)  # labels의 형태 확인

In [ ]:
trainer.fit(model, data_module)

In [ ]:
trainer.test(model, data_module)

- Predictions

In [ ]:
# 학습된 모델 로드
trained_model = Summarizer.load_from_checkpoint(
    trainer.checkpoint_callback.best_model_path
)
trained_model.eval()
trained_model.freeze()

In [ ]:
# 데이터 전처리 함수
def data_process(text):
    # 문장 분리
    sents = kss.split_sentences(text.replace('\n', ''))

    #데이터 가공
    tokenlist = [tokenizer(sent, add_special_tokens=True, max_length=MAX_TOKEN_COUNT, truncation=True) for sent in sents]

    src, segs, clss = [], [], []
    odd = 0

    for tkns in tokenlist:

        if odd > 1:
            odd = 0
        clss.append(len(src))
        src.extend(tkns['input_ids'])
        segs.extend([odd] * len(tkns['input_ids']))
        odd += 1

        #truncation
        if len(src) >= MAX_TOKEN_COUNT:
            src = src[:MAX_TOKEN_COUNT]  # 512개까지만 유지
            segs = segs[:MAX_TOKEN_COUNT]
            clss = clss[:MAX_TOKEN_COUNT]  # CLS 토큰도 동일하게 유지
            break  # 최대 길이를 초과하면 중단

    #padding
    pad_len = MAX_TOKEN_COUNT - len(src)
    src.extend([0] * pad_len)
    segs.extend([0] * pad_len)
    clss.extend([-1] * (MAX_TOKEN_COUNT - len(clss)))

    return dict(
        sents = sents, # 정답 출력을 위해 원문 문장 저장
        src = torch.tensor(src),
        segs = torch.tensor(segs),
        clss = torch.tensor(clss),
    )

In [ ]:
# 요약 수행 함수
def summarize_test(text):
    data = data_process(text.replace('\n', ''))

    # 입력 데이터를 모델과 동일한 장치(GPU)로 이동
    src = data['src'].unsqueeze(0).to(device)
    segs = data['segs'].unsqueeze(0).to(device)
    clss = data['clss'].unsqueeze(0).to(device)

    #trained_model에 넣어 결과값 반환
    _, rtn = trained_model(src, segs, clss)
    rtn = rtn.squeeze()

    # 예측 결과값을 받기 위한 프로세스
    rtn_sort, idx = rtn.sort(descending=True)

    rtn_sort = rtn_sort.tolist()
    idx = idx.tolist()

    # 0이 나오기 전까지 유효한 문장 선택
    end_idx = rtn_sort.index(0)

    rtn_sort = rtn_sort[:end_idx]
    idx = idx[:end_idx]

    # 최상위 3개 문장 선택
    if len(idx) > 3:
        rslt = idx[:3]
    else:
        rslt = idx

    summ = []
    print(' *** 입력한 문단의 요약문은 ...')
    for i, r in enumerate(rslt):
        summ.append(data['sents'][r])
        print('[', i+1, ']', summ[i])

    return summ

In [ ]:
#테스트 문장 입력
test_context = '''이재명 더불어민주당 대선후보는 26일 변호사비 대납 의혹과 관련, "내가 정말로 변호사비를 불법으로 받았으면 나를 구속하라"고 반박했다.
이 후보는 이날 오후 전남 신안군 응급의료 전용헬기 계류장에서 열린 '국민반상회' 후 기자들과 만나 한 시민단체 대표가 고액 수임료 의혹 증거라며 제시한 녹취록에 대해 "조작됐다는 증거를 갖고 있고 검찰에도 제출했다. 검찰과 수사기관들은 빨리 처리하시라"며 이같이 말했다.
앞서 이민구 깨어있는시민연대당 대표는 이 후보가 특정 변호사에게 수임료로 현금과 주식 등 20억원을 줬다는 의혹을 주장하며 녹취록을 제출한 바 있다. 이에 대해 송평수 선대위 부대변인은 "허위사실"이라며 "깨시민당 이 대표에게 제보를 했다는 시민단체 대표 이모 씨가 제3자로부터 기부금을 받아낼 목적으로 허위사실을 녹음한 후, 이 모 변호사에게까지 접근했다. 이러한 비상식적이고 악의적인 행태는 이재명 후보에 대한 정치적 타격을 가할 목적으로 치밀하게 준비한 것"이라고 반박했다.
이에 대해 이 후보는 "그것도 조직폭력배 조작에 버금가는 조작사건이라는 게 곧 드러날 것"이라며 "팩트확인을 하고 언급하면 좋겠다. 당사자도 아니고 제3자들이 자기끼리 녹음한 게 가치가 있느냐"고 반문했다.
그는 "사실이 아니면 무고하고 음해하는 사람들을 무고 혐의나 공직선거법 위반으로 빨리 처리해서 처벌하시라"며 "선거 국면에서 하루이틀도, 한두번도 아니고 '조폭이 뇌물 줬다'는 (허위사실 유포를) 왜 아직도 처리 안 하고 있느냐"고 검경에 불만을 드러냈다.
이어 "허위사실이 드러났으면 당연히 다시는 그런 일이 없게 해야 하는 것 아닌가. 이해가 안 된다"며 "선거관리, 또는 범죄를 단속하는 국가기관들이 이런 식으로 허위사실 유포나 무고 행위를 방치해 정치적 공격 수단으로 쓰게 하면 안 된다"고 했다.
이 후보는 또 자신이 구민주-동교동계와 접촉해 복당을 타진했다는 언론보도와 관련해선 "구체적으로 어떤 사람을 범주별로 나눠 무슨 계, 진영으로 말하는 것은 아니다"라며 "시점을 언젠가 정해 벌점이니, 제재니, 제한이니 다 없애고 모두가 합류할 수 있도록 할 생각"이라고 말했다.
종전에 언급했던 '대사면' 방침을 재확인한 셈이다. 그는 "민주당에 계셨던 분, 또 민주당에 있지 않았더라도 앞으로 함께할 분들에게 계속 연락을 하고 있다"며 "만나고 전화하고 힘을 합치자고 권유하고 있다"고 했다.
그는 " 현재 민주당이 이미 열린민주당과의 통합을 협의하고 있다"며 "거기에 더해서 꼭 민주계라고 말할 필요는 없고 부패사범이나 파렴치범으로 탈당하거나 또는 제명된 사람들이 아니라면, 국가의 미래를 걱정하는 민주개혁 진영의 일원이라면 가리지 말고 과거의 어떤 일이든 그러지 말고 힘을 합치자"고 강조했다.
언론보도에 따르면, 이 후보는 최근 구민주계인 정대철 전 고문과 연락을 주고 받으며 천정배, 정동영 전 의원 등 민주당을 탈당했던 옛 동교동계와 호남 인사들의 복당을 타진했다.
'''

In [ ]:
# test_context 요약
summary_context = summarize_test(test_context)

In [ ]:
test_text = """
현대자동차는 전기차 생산 확대를 위해 신규 공장을 건설한다고 밝혔다.
이 공장은 2026년까지 완공될 예정이며, 연간 50만 대의 전기차를 생산할 계획이다.
또한, 현대차는 배터리 성능 향상을 위한 연구 개발도 적극적으로 추진하고 있다.
"""

summary_result = summarize_test(test_text)

print("요약 결과:")
for idx, sentence in enumerate(summary_result, 1):
    print(f"{idx}. {sentence}")

NameError: name 'summarize_test' is not defined

- 세션 재시작 시 필수 실행 코드